In [19]:
from lumibot.brokers import Alpaca
from lumibot.backtesting import YahooDataBacktesting
from lumibot.strategies import Strategy
from lumibot.traders import Trader
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
# from alpaca_trade_api import REST
from utils import estimation_sentiment


In [2]:
#get the APIKEYS
load_dotenv()

API_KEY=os.getenv("Key")
API_SECRET=os.getenv("Secret")
BASE_URL=os.getenv("Endpoint")


In [3]:
AlpacaConfig = {
    "API_KEY": API_KEY,
    "API_SECRET": API_SECRET,
    "PAPER": True
}

In [4]:
broker=Alpaca(AlpacaConfig)

2024-07-18 13:38:19 | alpaca.trading.stream | INFO | started trading stream
2024-07-18 13:38:19 | asyncio | INFO | [unknown] Waiting for the socket stream connection to be established, 
                method _stream_established must be called
2024-07-18 13:38:19 | alpaca.trading.stream | INFO | starting trading websocket connection


2024-07-18 13:38:22 | alpaca.trading.stream | INFO | connected to: BaseURL.TRADING_STREAM_PAPER


In [7]:
# class MLTrader(Strategy):
#     def initialize(self, symbol="SPY", cash_at_risk=.5):
#         self.symbol=symbol
#         #the we determine how often we wanna make a trader by sleeptime
#         self.sleeptime="24H"
#         #then set where to start with the trader in every 24 hours
#         self.last_trade = None
#         self.cash_at_risk=cash_at_risk
#         self.api=REST(base_url=BASE_URL, key_id=API_KEY, secret_key=API_SECRET)
    

#     #position sizing and limiting
#     def position_size(self):
#         #making sure that we have good use of money when trading
#         cash_left_in_account = self.get_cash()
#         last_price = self.get_last_price(self.symbol)
#         quantity = round(cash_left_in_account*self.cash_at_risk/last_price, 0)
#         return cash_left_in_account, last_price, quantity
    

#     #get dates function
#     def get_dates(self):
#         today=self.get_datetime()
#         three_days_ago = today - timedelta(days=3)
#         return today.strftime('%Y-%m-%d'), three_days_ago.strftime('%Y-%m-%d')
    

#     #getting the news
#     def get_news_sentiment(self):
#        today, three_days_ago = self.get_dates()
#        news=self.api.get_news(symbol="SPY", start=three_days_ago, end=today)
#        news = [ev.__dict__["_raw"]["headline"] for ev in news]

#        probability, sentiment = estimation_sentiment(news)

#        return probability, sentiment

#     def trading_process(self):
#         #get the returns in position_size function
#         cash_left_in_account, last_price, quantity = self.position_size()

#         if cash_left_in_account > last_price:
#             probability =self.get_news_sentiment()
#             sentiment = self.get_news_sentiment()
#             # print(probability, sentiment)
#             if probability [0] > .999 and sentiment == "positive":
#                 if self.last_trade == "sell":
#                     self.sell_all()

#                 new_order = self.create_order(
#                     self.symbol,
#                     quantity,
#                     "buy",
#                     type="bracket",
#                     take_profit_price=last_price*1.20,
#                     stop_loss_price=last_price*0.95,
#                 )
#                 self.submit_order(new_order)
#                 self.last_trade="buy"
#             if probability [0] > .999 and sentiment == "negative":
#                 if self.last_trade == "buy":
#                     self.sell_all()

#                 new_order = self.create_order(
#                     self.symbol,
#                     quantity,
#                     "sell ",
#                     type="bracket",
#                     take_profit_price=last_price*.8,
#                     stop_loss_price=last_price*1.05,
#                 )
#                 self.submit_order(new_order)
#                 self.last_trade="sell"


In [20]:
class Trader(Strategy):
    def initialize(self, symbol="VOO", cash_risk=.5):
        self.symbol=symbol
        #the we determine how often we wanna make a trader by sleeptime
        self.sleeptime="24H"
        #then set where to start with the trader in every 24 hours
        self.last_trade = None
        self.cash_risk=cash_risk
        # self.api=REST(base_url=BASE_URL, key_id=API_KEY, secret_key=API_SECRET)
    

    #position sizing and limiting
    def position_size(self):
        #making sure that we have good use of money when trading
        cash_left_in_account = self.get_cash()
        last_price = self.get_last_price(self.symbol)
        quantity = round(cash_left_in_account*self.cash_risk/last_price, 0)
        return cash_left_in_account, last_price, quantity
    

    #get dates function
    def get_dates(self):
        today=self.get_datetime()
        three_days_ago = today - timedelta(days=3)
        return today.strftime('%Y-%m-%d'), three_days_ago.strftime('%Y-%m-%d')
    

    #getting the news
    def get_news_sentiment(self):
       today, three_days_ago = self.get_dates()
       news=self.api.get_news(symbol="VOO", start=three_days_ago, end=today)
       news = [ev.__dict__["_raw"]["headline"] for ev in news]

       probability, sentiment = estimation_sentiment(news)

       return probability, sentiment

    def trading_process(self):
        #get the returns in position_size function
        cash_left_in_account, last_price, quantity = self.position_size()

        if cash_left_in_account > last_price:
            if self.last_trade == None:
                new_order = self.create_order(
                    self.symbol,
                    quantity,
                    "buy",
                    type="bracket",
                    take_price=last_price*1.50,
                    stop_loss_price=last_price*0.95,
                )
                self.submit_order(new_order)
                self.last_trade="buy"

In [21]:
#lets set up the trading instance with our strategy variable
strategy=Trader(
    name="mlstrat",
    broker=broker,
    parameters={
        "symbol": "VOO",
        "cash_at_risk":.5
    }
)

In [22]:
backtesting_start_date=datetime(2023,12,15)
backtesting_end_date=datetime(2023,12,31)

"""
step 3: set up the backtesting with the data source
the data source here is YahooBackTesting class but if
you have your own data, you would typically use panda backtesting

The backtesting_params may include the budget and other stuff too
check here ```https://pypi.org/project/lumibot/1.3.2/```
"""
strategy.backtest(
    YahooDataBacktesting,
    backtesting_start_date,
    backtesting_end_date,
    parameters={
        "symbol": "VOO",
        "cash_at_risk": .5
    }
)


Starting backtest for Trader...
Progress || 100.00%  [Elapsed: 0:00:01 ETA: 0:00:00] Portfolio Val: 100,000.00
Creating trades plot...

Creating indicators plot...

Creating tearsheet...
2024-07-18 14:12:29,863: root: ERROR: Not enough data to create a tearsheet, at least 2 days of data are required. Skipping


{'cagr': 0.0,
 'volatility': 0.0,
 'sharpe': 0,
 'max_drawdown': {'drawdown': 0.0,
  'date': Timestamp('2023-12-16 09:30:00-0500', tz='America/New_York')},
 'romad': 0,
 'total_return': 0.0}